In [28]:
import numpy as np
from scipy.io import wavfile

In [29]:
def f(x):
    f_0 = 1
    envelope = lambda x: np.exp(-x)
    return np.sin(x * np.pi * 2 * f_0) * envelope(x)

def ACF(f, W, t,lag ):
    return np.sum(f[t:W + t] * f[lag + t: lag + W + t])

def DF(f, W, t, lag):
    return ACF(f, W, t, 0) + ACF(f, W, t + lag, 0) - (2 * ACF(f, W, t, lag))

def CMNDF(f, W, t, lag):
    if lag == 0:
        return 1
    return DF(f, W, t, lag) / np.sum([DF(f, W, t, j + 1) for j in range(lag)]) * lag

def detect_pitch(f, W, t, sample_rate, bounds, thresh=0.1):
    CMNDF_vals= [CMNDF(f, W, t, i) for i in range(*bounds)]
    sample = None
    for i, val in enumerate(CMNDF_vals):
        if val < thresh:
            sample += i + bounds[0]
            break
    if sample is None:
        sample = np.argmin(CMNDF_vals) + bounds[0]

    return sample_rate / sample

In [30]:
# TODO: test on larger data set
# this doesn't work right now for some reason
sample_rate, data = wavfile.read("test.wav") 
data = data.astype(np.float64)
window_size = int(5 / 2000 * 44100)
bounds = [20, 2000]

pitches = []
for i in range(data.shape[0] // (window_size + 3)):
    pitches.append(
        detect_pitch(
            data,
            window_size,
            i * window_size,
            sample_rate,
            bounds
        )
    )

print(pitches)

/var/folders/np/tqrpvjw547x9313tqpr0sfd00000gn/T/ipykernel_63173/2641022657.py:15: RuntimeWarning: invalid value encountered in scalar divide
  return DF(f, W, t, lag) / np.sum([DF(f, W, t, j + 1) for j in range(lag)]) * lag


TypeError: unsupported operand type(s) for +=: 'NoneType' and 'int'

In [31]:
sample_rate = 500 
start = 0
end = 5
num_samples = int(sample_rate * (end - start) + 1)
window_size = 200 
bounds = [20, num_samples // 2]

x = np.linspace(start, end, num_samples)
print(detect_pitch(f(x), window_size, 1, sample_rate, bounds))

1.002004008016032


In [34]:
bounds = [10, 20]
[i for i in range(*bounds)]

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]